
### Steps To Be Followed

    1. Importing necessary Libraries
    2. Creating S3 bucket
    3. Mapping train And Test Data in S3
    4. Mapping The path of the models in S3



In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

### Creating S3 bucket

In [2]:
bucket_name = 'bankapplicaton'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-1


In [4]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
        print('S3 bucket created successfully.')
except Exception as e:
    print("S3 ERROR ::: ",e)

S3 bucket created successfully.


In [5]:
prefix = "xgboost-as-a-built-in-algo"
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplicaton/xgboost-as-a-built-in-algo/output


### Downloading The Dataset And Storing in S3

In [6]:
import pandas as pd
import urllib

DATASET_URL = "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv"
try:
    urllib.request.urlretrieve(DATASET_URL, "bank_clean.csv")
    print("Success : Downloaded bank_clean.csv")
except Exception as e:
    print("Data Load Error : ",e)
    
try:
    model_data = pd.read_csv("./bank_clean.csv",index_col=0)
    print("Success : Data loaded into dataframe")
except Exception as e:
    print('Data load error: ',e)

Success : Downloaded bank_clean.csv
Success : Data loaded into dataframe


In [8]:
model_data.head(3)

,age,campaign,pdays,previous,no_previous_contact,not_working,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,marital_unknown,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,default_no,default_unknown,default_yes,housing_no,housing_unknown,housing_yes,loan_no,loan_unknown,loan_yes,contact_cellular,contact_telephone,month_apr,month_aug,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,56,1,999,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
1,57,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
2,37,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0


In [10]:
### TRAIN TEST SPLIT

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1,random_state=1729), [int(0.7*len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [12]:
### Saving Train And Test Into Buckets
## We start with Train Data

import os
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'],
                                               axis=1)],
                                               axis=1).to_csv("train.csv", index=False,header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data = 's3://{}/{}/train'.format(bucket_name,prefix),content_type='csv')

In [14]:
### TEST DATA INTO BUCKET
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'],
                                               axis=1)],
                                               axis=1).to_csv("test.csv", index=False,header=False)

boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix,'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data = 's3://{}/{}/test'.format(bucket_name,prefix),content_type='csv')

### BUILDING MODELS AND TRAINING Xgboost - INBUILT ALGORITHM

In [16]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = get_image_uri(boto3.Session().region_name,
                          'xgboost', repo_version='1.0-1')



The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [17]:
# initialize hyperparameters

hyperparameters = {
    "max_depth": "5",
    "eta": "0.2",
    "gamma":"4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective":"binary:logistic",
    "num_round": 50
}

In [19]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri = container,
                                         hyperparameters=hyperparameters,
                                         role=sagemaker.get_execution_role(),
                                         train_instance_count=1,
                                         train_instance_type="ml.m5.2xlarge",
                                         train_volume_size=5, #5 GB
                                         output_path=output_path,
                                         train_use_spot_instances=True,
                                         train_max_run=300,
                                         train_max_wait=600)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_run has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_use_spot_instances has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_max_wait has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_volume_size has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [21]:
estimator.fit({'train':s3_input_train,'validation': s3_input_test})

2022-11-20 20:27:12 Starting - Starting the training job...
2022-11-20 20:27:38 Starting - Preparing the instances for trainingProfilerReport-1668976031: InProgress
.........
2022-11-20 20:29:00 Downloading - Downloading input data...
2022-11-20 20:29:39 Training - Downloading the training image...
2022-11-20 20:30:06 Uploading - Uploading generated training modelINFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[20:29:59] 12357x59 matrix with 729063 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delim

### Deploy Machine Learning Model As Endpoints

In [22]:
xgb_predictor = estimator.deploy(initial_instance_count=1,
                                instance_type='ml.m4.xlarge')

---------!

### Prediction of the Test Data

In [24]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
# xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


(12357,)


In [25]:
predictions_array

array([0.06547958, 0.052804  , 0.06467394, ..., 0.03337643, 0.04640768,
       0.04087697])

In [26]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.1%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10813)    28% (123)
Purchase        9% (1104)     72% (317) 



### Deleting The Endpoints

In [27]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[{'ResponseMetadata': {'RequestId': 'SZBYBFZFWHHSJH7T',
   'HostId': 'Y84jSCuflk0H3RcNP9bK1Cgd3YuGY/uYp+KX5xtXjTWSL3GjNOf70pIBwltu80YfOmVjYJmR1y0=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Y84jSCuflk0H3RcNP9bK1Cgd3YuGY/uYp+KX5xtXjTWSL3GjNOf70pIBwltu80YfOmVjYJmR1y0=',
    'x-amz-request-id': 'SZBYBFZFWHHSJH7T',
    'date': 'Sun, 20 Nov 2022 20:39:37 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-11-20-20-27-11-968/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-11-20-20-27-11-968/rule-output/ProfilerReport-1668976031/profiler-output/profiler-reports/StepOutlier.json'},
   {'Key': 'xgboost-as-a-built-in-algo/output/sagemaker-xgboost-2022-11-20-20-27-11-968/rule-output/ProfilerReport-1668976031/profile